In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
from test_learner2d import *

In [ ]:
adaptive.notebook_extension()

#### define bounds and peaks

In [ ]:
# relevant parameters
discovery_factor = 50 # how difficult to find are the peaks
min_resolution = 2 # minimum resolution 2 means calculate first only the bounds
num_peaks = 5

In [ ]:
bounds = [(-1, 1), (-1, 1)]
total_area = (bounds[0][1] - bounds[0][0]) * (bounds[1][1] - bounds[1][0])
np.random.seed(0)

min_area = (total_area / min_resolution ** 2) / discovery_factor
radius = (min_area / np.pi) ** 0.5
tol = min_area / 2
# random peaks
box = (1-4*radius) # ensure peaks inside the area
xy0 = np.ndarray((num_peaks, 2))
for i in range(num_peaks):
    xy0[i] = (2*np.random.rand(2) - 1) * box

#### test func = cauchy

In [ ]:
# for func in [cauchy(xy0, radius), normal(xy0, radius)]:
func = partial(cauchy, xy0=xy0, radius=radius)
learner = adaptive.learner.Learner2D(func, bounds=bounds)

runner = adaptive.Runner(learner, executor=executor,
                        goal=lambda l: l.loss() < tol)
sleep(1)

In [ ]:
if runner.task.done() is False:
    sleep(0.5)

In [ ]:
found_peaks = learner.unscale(peak_finder(learner.ip()))
# peaks found can be nearly degenerate, and the number can be
# greater than num_peaks
# The number of peaks found can be smaller than num_peaks, if some
# peak in xy0 is nearly degenerate, or if some peak is not found,
# in which case the test fails
deviations = np.ndarray(num_peaks)
for i in range(num_peaks):
    deviations[i] = np.linalg.norm(xy0[i] - found_peaks, axis=-1).min()

In [ ]:
%%opts Scatter (size=5)
# density plot, with blue dots as real peaks, and red dots as found peaks
learner.plot() * hv.Scatter(xy0) * hv.Scatter((learner.unscale(peak_finder(learner.ip()))))

In [ ]:
assert np.all(deviations < radius)

In [ ]:
executor.shutdown()

#### test func = normal

In [ ]:
executor = ProcessPoolExecutor(max_workers=1)

In [ ]:
def normal(xy, xy0, radius):
    xy0 = np.atleast_2d(xy0)
    sigma = radius / 2
    num_peaks = xy0.shape[0]
    x, y = xy
    result = 0
    for i in range(num_peaks):
        x0, y0 = xy0[i]
        result += (1 / (2 * np.pi * sigma**2) *
                np.exp(-((x-x0) ** 2 + (y-y0) ** 2) /
                        (2 * sigma**2)))
    return result

In [ ]:
# for func in [cauchy(xy0, radius), normal(xy0, radius)]:
func = partial(normal, xy0=xy0, radius=radius)
learner = adaptive.learner.Learner2D(func, bounds=bounds)

runner = adaptive.Runner(learner, executor=executor,
                        goal=lambda l: l.loss() < tol)
sleep(1)

In [ ]:
if runner.task.done() is False:
    sleep(0.5)

In [ ]:
found_peaks = learner.unscale(peak_finder(learner.ip()))
# peaks found can be nearly degenerate, and the number can be
# greater than num_peaks
# The number of peaks found can be smaller than num_peaks, if some
# peak in xy0 is nearly degenerate, or if some peak is not found,
# in which case the test fails
deviations = np.ndarray(num_peaks)
for i in range(num_peaks):
    deviations[i] = np.linalg.norm(xy0[i] - found_peaks, axis=-1).min()

In [ ]:
%%opts Scatter (size=5)
# density plot, with blue dots as real peaks, and red dots as found peaks
learner.plot() * hv.Scatter(xy0) * hv.Scatter((learner.unscale(peak_finder(learner.ip()))))

In [ ]:
assert np.all(deviations < radius)

In [ ]:
executor.shutdown()